In [1]:
from pathlib import Path
from malid import config, logger
from malid.train import train_metamodel
from malid.datamodels import (
    GeneLocus,
    TargetObsColumnEnum,
)
import pandas as pd
from IPython.display import display, Markdown
from typing import List

In [2]:
base_model_train_fold_name = "train_smaller"
metamodel_fold_label_train = "validation"

In [3]:
models_of_interest = [
    "rf_multiclass",
    "xgboost",
    "lasso_multiclass",
    "lasso_cv",
    "ridge_cv",
    "elasticnet_cv",
    "linearsvm_ovr",
]
models_of_interest

['rf_multiclass',
 'xgboost',
 'lasso_multiclass',
 'lasso_cv',
 'ridge_cv',
 'elasticnet_cv',
 'linearsvm_ovr']

In [4]:
def choose(gene_locus: GeneLocus, classification_targets: List[TargetObsColumnEnum]):
    for target_obs_column in classification_targets:
        try:
            flavors = train_metamodel.get_metamodel_flavors(
                gene_locus=gene_locus,
                target_obs_column=target_obs_column,
                fold_id=config.all_fold_ids[0],
                base_model_train_fold_name=base_model_train_fold_name,
            )
        except Exception as err:
            logger.warning(
                f"Failed to generate metamodel flavors for {gene_locus}, {target_obs_column}: {err}"
            )
            continue

        for metamodel_flavor, metamodel_config in flavors.items():
            _output_suffix = (
                Path(gene_locus.name)
                / target_obs_column.name
                / metamodel_flavor
                / f"{base_model_train_fold_name}_applied_to_{metamodel_fold_label_train}_model"
            )
            results_output_prefix = (
                config.paths.second_stage_blending_metamodel_output_dir / _output_suffix
            )

            display(
                Markdown(
                    f"# {gene_locus}, {target_obs_column}, metamodel flavor {metamodel_flavor}"
                )
            )

            try:
                ## All results in a table
                df = pd.read_csv(
                    f"{results_output_prefix}.compare_model_scores.test_set_performance.tsv",
                    sep="\t",
                    index_col=0,
                )
                df["missing_classes"] = df["missing_classes"].map(
                    {False: "✅ no", True: "❗ Missing classes!"}
                )
                df = df.loc[df.index.intersection(models_of_interest)].sort_values(
                    "ROC-AUC (weighted OvO) per fold", ascending=False
                )
                if "Accuracy global with abstention" in df.columns:
                    df = df[
                        [
                            "ROC-AUC (weighted OvO) per fold",
                            "au-PRC (weighted OvO) per fold",
                            "Accuracy global with abstention",
                            "MCC global with abstention",
                            "abstention_rate",
                            "sample_size including abstentions",
                            "n_abstentions",
                            "missing_classes",
                        ]
                    ]
                else:
                    df = df[
                        [
                            "ROC-AUC (weighted OvO) per fold",
                            "au-PRC (weighted OvO) per fold",
                            "Accuracy global",
                            "MCC global",
                            "abstention_rate",
                            "sample_size including abstentions",
                            "n_abstentions",
                            "missing_classes",
                        ]
                    ]
                display(df)
            except Exception as err:
                logger.warning(
                    f"{gene_locus}, {target_obs_column} flavor '{metamodel_flavor}': not yet run: {err}"
                )
                continue

In [5]:
for single_gene_locus in config.gene_loci_used:
    choose(single_gene_locus, config.classification_targets)

# GeneLocus.BCR, TargetObsColumnEnum.disease, metamodel flavor default

,ROC-AUC (weighted OvO) per fold,au-PRC (weighted OvO) per fold,Accuracy global with abstention,MCC global with abstention,abstention_rate,sample_size including abstentions,n_abstentions,missing_classes
linearsvm_ovr,0.964 +/- 0.005 (in 3 folds),0.963 +/- 0.006 (in 3 folds),0.835,0.762,0.022917,480,11,✅ no
lasso_multiclass,0.960 +/- 0.006 (in 3 folds),0.959 +/- 0.008 (in 3 folds),0.827,0.753,0.022917,480,11,✅ no
rf_multiclass,0.959 +/- 0.009 (in 3 folds),0.954 +/- 0.014 (in 3 folds),0.831,0.755,0.022917,480,11,✅ no
elasticnet_cv,0.957 +/- 0.008 (in 3 folds),0.958 +/- 0.009 (in 3 folds),0.802,0.713,0.022917,480,11,✅ no
xgboost,0.953 +/- 0.005 (in 3 folds),0.951 +/- 0.009 (in 3 folds),0.812,0.728,0.022917,480,11,✅ no
lasso_cv,0.949 +/- 0.005 (in 3 folds),0.954 +/- 0.007 (in 3 folds),0.800,0.709,0.022917,480,11,✅ no
ridge_cv,0.948 +/- 0.005 (in 3 folds),0.951 +/- 0.006 (in 3 folds),0.802,0.713,0.022917,480,11,✅ no


# GeneLocus.BCR, TargetObsColumnEnum.disease, metamodel flavor isotype_counts_only

,ROC-AUC (weighted OvO) per fold,au-PRC (weighted OvO) per fold,Accuracy global,MCC global,abstention_rate,sample_size including abstentions,n_abstentions,missing_classes
elasticnet_cv,0.684 +/- 0.020 (in 3 folds),0.685 +/- 0.026 (in 3 folds),0.494,0.181,0.0,480,0,❗ Missing classes!
lasso_cv,0.676 +/- 0.023 (in 3 folds),0.681 +/- 0.032 (in 3 folds),0.508,0.215,0.0,480,0,✅ no
ridge_cv,0.675 +/- 0.022 (in 3 folds),0.672 +/- 0.018 (in 3 folds),0.494,0.183,0.0,480,0,❗ Missing classes!
linearsvm_ovr,0.674 +/- 0.020 (in 3 folds),0.666 +/- 0.020 (in 3 folds),0.477,0.194,0.0,480,0,✅ no
rf_multiclass,0.673 +/- 0.030 (in 3 folds),0.648 +/- 0.029 (in 3 folds),0.515,0.259,0.0,480,0,✅ no
lasso_multiclass,0.668 +/- 0.020 (in 3 folds),0.655 +/- 0.014 (in 3 folds),0.471,0.223,0.0,480,0,✅ no
xgboost,0.644 +/- 0.020 (in 3 folds),0.629 +/- 0.019 (in 3 folds),0.483,0.218,0.0,480,0,✅ no


# GeneLocus.BCR, TargetObsColumnEnum.disease_all_demographics_present, metamodel flavor default

...

,ROC-AUC (weighted OvO) per fold,au-PRC (weighted OvO) per fold,Accuracy global,MCC global,abstention_rate,sample_size including abstentions,n_abstentions,missing_classes
linearsvm_ovr,0.992 +/- 0.006 (in 3 folds),0.998 +/- 0.002 (in 3 folds),0.944,0.851,0.0,252,0,✅ no
lasso_multiclass,0.992 +/- 0.005 (in 3 folds),0.998 +/- 0.001 (in 3 folds),0.944,0.851,0.0,252,0,✅ no
ridge_cv,0.992 +/- 0.004 (in 3 folds),0.998 +/- 0.001 (in 3 folds),0.913,0.742,0.0,252,0,✅ no
elasticnet_cv,0.992 +/- 0.003 (in 3 folds),0.998 +/- 0.001 (in 3 folds),0.933,0.804,0.0,252,0,✅ no
lasso_cv,0.988 +/- 0.009 (in 3 folds),0.997 +/- 0.003 (in 3 folds),0.937,0.816,0.0,252,0,✅ no
rf_multiclass,0.986 +/- 0.009 (in 3 folds),0.996 +/- 0.002 (in 3 folds),0.944,0.843,0.0,252,0,✅ no
xgboost,0.981 +/- 0.012 (in 3 folds),0.994 +/- 0.004 (in 3 folds),0.933,0.811,0.0,252,0,✅ no


# GeneLocus.TCR, TargetObsColumnEnum.hiv_vs_healthy, metamodel flavor default

,ROC-AUC (weighted OvO) per fold,au-PRC (weighted OvO) per fold,Accuracy global with abstention,MCC global with abstention,abstention_rate,sample_size including abstentions,n_abstentions,missing_classes
linearsvm_ovr,0.937 +/- 0.006 (in 3 folds),0.973 +/- 0.004 (in 3 folds),0.866,0.723,0.017123,292,5,✅ no
lasso_cv,0.933 +/- 0.011 (in 3 folds),0.972 +/- 0.006 (in 3 folds),0.808,0.563,0.017123,292,5,✅ no
lasso_multiclass,0.933 +/- 0.010 (in 3 folds),0.973 +/- 0.005 (in 3 folds),0.853,0.693,0.017123,292,5,✅ no
elasticnet_cv,0.929 +/- 0.017 (in 3 folds),0.972 +/- 0.007 (in 3 folds),0.798,0.540,0.017123,292,5,✅ no
rf_multiclass,0.925 +/- 0.031 (in 3 folds),0.966 +/- 0.016 (in 3 folds),0.818,0.605,0.017123,292,5,✅ no
ridge_cv,0.924 +/- 0.025 (in 3 folds),0.970 +/- 0.010 (in 3 folds),0.805,0.558,0.017123,292,5,✅ no
xgboost,0.922 +/- 0.032 (in 3 folds),0.957 +/- 0.029 (in 3 folds),0.836,0.645,0.017123,292,5,✅ no


# GeneLocus.TCR, TargetObsColumnEnum.lupus_vs_healthy, metamodel flavor default

,ROC-AUC (weighted OvO) per fold,au-PRC (weighted OvO) per fold,Accuracy global with abstention,MCC global with abstention,abstention_rate,sample_size including abstentions,n_abstentions,missing_classes
elasticnet_cv,0.973 +/- 0.012 (in 3 folds),0.940 +/- 0.017 (in 3 folds),0.888,0.693,0.01938,258,5,✅ no
lasso_multiclass,0.972 +/- 0.011 (in 3 folds),0.939 +/- 0.016 (in 3 folds),0.888,0.731,0.01938,258,5,✅ no
lasso_cv,0.971 +/- 0.016 (in 3 folds),0.933 +/- 0.030 (in 3 folds),0.884,0.681,0.01938,258,5,✅ no
ridge_cv,0.971 +/- 0.014 (in 3 folds),0.932 +/- 0.030 (in 3 folds),0.872,0.643,0.01938,258,5,✅ no
linearsvm_ovr,0.971 +/- 0.012 (in 3 folds),0.940 +/- 0.015 (in 3 folds),0.880,0.712,0.01938,258,5,✅ no
rf_multiclass,0.959 +/- 0.023 (in 3 folds),0.916 +/- 0.035 (in 3 folds),0.895,0.723,0.01938,258,5,✅ no
xgboost,0.948 +/- 0.034 (in 3 folds),0.912 +/- 0.040 (in 3 folds),0.899,0.730,0.01938,258,5,✅ no


In [6]:
if len(config.gene_loci_used) > 1:
    choose(config.gene_loci_used, config.classification_targets)

# GeneLocus.BCR|TCR, TargetObsColumnEnum.disease, metamodel flavor default

,ROC-AUC (weighted OvO) per fold,au-PRC (weighted OvO) per fold,Accuracy global with abstention,MCC global with abstention,abstention_rate,sample_size including abstentions,n_abstentions,missing_classes
lasso_multiclass,0.983 +/- 0.005 (in 3 folds),0.980 +/- 0.006 (in 3 folds),0.879,0.826,0.016908,414,7,✅ no
elasticnet_cv,0.982 +/- 0.005 (in 3 folds),0.979 +/- 0.006 (in 3 folds),0.884,0.830,0.016908,414,7,✅ no
ridge_cv,0.982 +/- 0.005 (in 3 folds),0.976 +/- 0.008 (in 3 folds),0.877,0.819,0.016908,414,7,✅ no
rf_multiclass,0.981 +/- 0.013 (in 3 folds),0.976 +/- 0.016 (in 3 folds),0.886,0.833,0.016908,414,7,✅ no
linearsvm_ovr,0.980 +/- 0.003 (in 3 folds),0.977 +/- 0.005 (in 3 folds),0.884,0.832,0.016908,414,7,✅ no
lasso_cv,0.976 +/- 0.010 (in 3 folds),0.975 +/- 0.007 (in 3 folds),0.882,0.826,0.016908,414,7,✅ no
xgboost,0.973 +/- 0.008 (in 3 folds),0.971 +/- 0.008 (in 3 folds),0.874,0.817,0.016908,414,7,✅ no


# GeneLocus.BCR|TCR, TargetObsColumnEnum.disease, metamodel flavor isotype_counts_only

,ROC-AUC (weighted OvO) per fold,au-PRC (weighted OvO) per fold,Accuracy global,MCC global,abstention_rate,sample_size including abstentions,n_abstentions,missing_classes
lasso_cv,0.707 +/- 0.008 (in 3 folds),0.713 +/- 0.017 (in 3 folds),0.543,0.261,0.0,414,0,❗ Missing classes!
lasso_multiclass,0.704 +/- 0.023 (in 3 folds),0.697 +/- 0.014 (in 3 folds),0.500,0.268,0.0,414,0,✅ no
rf_multiclass,0.702 +/- 0.019 (in 3 folds),0.684 +/- 0.017 (in 3 folds),0.548,0.302,0.0,414,0,✅ no
elasticnet_cv,0.702 +/- 0.003 (in 3 folds),0.703 +/- 0.017 (in 3 folds),0.539,0.252,0.0,414,0,❗ Missing classes!
linearsvm_ovr,0.695 +/- 0.020 (in 3 folds),0.686 +/- 0.015 (in 3 folds),0.502,0.234,0.0,414,0,✅ no
ridge_cv,0.691 +/- 0.022 (in 3 folds),0.683 +/- 0.023 (in 3 folds),0.522,0.217,0.0,414,0,❗ Missing classes!
xgboost,0.667 +/- 0.013 (in 3 folds),0.662 +/- 0.017 (in 3 folds),0.512,0.256,0.0,414,0,✅ no


# GeneLocus.BCR|TCR, TargetObsColumnEnum.disease_all_demographics_present, metamodel flavor default

...

,ROC-AUC (weighted OvO) per fold,au-PRC (weighted OvO) per fold,Accuracy global with abstention,MCC global with abstention,abstention_rate,sample_size including abstentions,n_abstentions,missing_classes
elasticnet_cv,0.999 +/- 0.001 (in 3 folds),1.000 +/- 0.000 (in 3 folds),0.948,0.854,0.015873,252,4,✅ no
ridge_cv,0.999 +/- 0.001 (in 3 folds),1.000 +/- 0.000 (in 3 folds),0.944,0.842,0.015873,252,4,✅ no
lasso_cv,0.998 +/- 0.002 (in 3 folds),1.000 +/- 0.000 (in 3 folds),0.956,0.878,0.015873,252,4,✅ no
lasso_multiclass,0.996 +/- 0.005 (in 3 folds),0.999 +/- 0.001 (in 3 folds),0.956,0.884,0.015873,252,4,✅ no
linearsvm_ovr,0.995 +/- 0.008 (in 3 folds),0.999 +/- 0.002 (in 3 folds),0.956,0.882,0.015873,252,4,✅ no
rf_multiclass,0.995 +/- 0.007 (in 3 folds),0.998 +/- 0.002 (in 3 folds),0.952,0.867,0.015873,252,4,✅ no
xgboost,0.990 +/- 0.008 (in 3 folds),0.997 +/- 0.003 (in 3 folds),0.940,0.834,0.015873,252,4,✅ no


# GeneLocus.BCR|TCR, TargetObsColumnEnum.hiv_vs_healthy, metamodel flavor default

,ROC-AUC (weighted OvO) per fold,au-PRC (weighted OvO) per fold,Accuracy global with abstention,MCC global with abstention,abstention_rate,sample_size including abstentions,n_abstentions,missing_classes
lasso_multiclass,0.989 +/- 0.005 (in 3 folds),0.995 +/- 0.002 (in 3 folds),0.925,0.842,0.034247,292,10,✅ no
elasticnet_cv,0.989 +/- 0.005 (in 3 folds),0.995 +/- 0.002 (in 3 folds),0.904,0.792,0.034247,292,10,✅ no
ridge_cv,0.987 +/- 0.007 (in 3 folds),0.994 +/- 0.002 (in 3 folds),0.928,0.848,0.034247,292,10,✅ no
linearsvm_ovr,0.987 +/- 0.004 (in 3 folds),0.993 +/- 0.003 (in 3 folds),0.918,0.828,0.034247,292,10,✅ no
lasso_cv,0.986 +/- 0.005 (in 3 folds),0.993 +/- 0.003 (in 3 folds),0.908,0.800,0.034247,292,10,✅ no
rf_multiclass,0.982 +/- 0.009 (in 3 folds),0.992 +/- 0.003 (in 3 folds),0.925,0.842,0.034247,292,10,✅ no
xgboost,0.978 +/- 0.006 (in 3 folds),0.991 +/- 0.002 (in 3 folds),0.911,0.817,0.034247,292,10,✅ no


# GeneLocus.BCR|TCR, TargetObsColumnEnum.lupus_vs_healthy, metamodel flavor default

,ROC-AUC (weighted OvO) per fold,au-PRC (weighted OvO) per fold,Accuracy global with abstention,MCC global with abstention,abstention_rate,sample_size including abstentions,n_abstentions,missing_classes
elasticnet_cv,0.982 +/- 0.015 (in 3 folds),0.958 +/- 0.033 (in 3 folds),0.891,0.710,0.031008,258,8,✅ no
lasso_multiclass,0.980 +/- 0.015 (in 3 folds),0.953 +/- 0.033 (in 3 folds),0.888,0.721,0.031008,258,8,✅ no
ridge_cv,0.979 +/- 0.017 (in 3 folds),0.953 +/- 0.035 (in 3 folds),0.891,0.706,0.031008,258,8,✅ no
lasso_cv,0.975 +/- 0.020 (in 3 folds),0.946 +/- 0.042 (in 3 folds),0.903,0.749,0.031008,258,8,✅ no
rf_multiclass,0.974 +/- 0.026 (in 3 folds),0.950 +/- 0.039 (in 3 folds),0.903,0.741,0.031008,258,8,✅ no
linearsvm_ovr,0.974 +/- 0.020 (in 3 folds),0.941 +/- 0.042 (in 3 folds),0.884,0.717,0.031008,258,8,✅ no
xgboost,0.969 +/- 0.032 (in 3 folds),0.937 +/- 0.053 (in 3 folds),0.903,0.746,0.031008,258,8,✅ no


In [7]:
# Default
for target in [
    TargetObsColumnEnum.disease,
]:
    for gene_locus in config.gene_loci_used:
        choose(gene_locus, [target])
    choose(config.gene_loci_used, [target])

# GeneLocus.BCR, TargetObsColumnEnum.disease, metamodel flavor default

,ROC-AUC (weighted OvO) per fold,au-PRC (weighted OvO) per fold,Accuracy global with abstention,MCC global with abstention,abstention_rate,sample_size including abstentions,n_abstentions,missing_classes
linearsvm_ovr,0.964 +/- 0.005 (in 3 folds),0.963 +/- 0.006 (in 3 folds),0.835,0.762,0.022917,480,11,✅ no
lasso_multiclass,0.960 +/- 0.006 (in 3 folds),0.959 +/- 0.008 (in 3 folds),0.827,0.753,0.022917,480,11,✅ no
rf_multiclass,0.959 +/- 0.009 (in 3 folds),0.954 +/- 0.014 (in 3 folds),0.831,0.755,0.022917,480,11,✅ no
elasticnet_cv,0.957 +/- 0.008 (in 3 folds),0.958 +/- 0.009 (in 3 folds),0.802,0.713,0.022917,480,11,✅ no
xgboost,0.953 +/- 0.005 (in 3 folds),0.951 +/- 0.009 (in 3 folds),0.812,0.728,0.022917,480,11,✅ no
lasso_cv,0.949 +/- 0.005 (in 3 folds),0.954 +/- 0.007 (in 3 folds),0.800,0.709,0.022917,480,11,✅ no
ridge_cv,0.948 +/- 0.005 (in 3 folds),0.951 +/- 0.006 (in 3 folds),0.802,0.713,0.022917,480,11,✅ no


# GeneLocus.BCR, TargetObsColumnEnum.disease, metamodel flavor isotype_counts_only

,ROC-AUC (weighted OvO) per fold,au-PRC (weighted OvO) per fold,Accuracy global,MCC global,abstention_rate,sample_size including abstentions,n_abstentions,missing_classes
elasticnet_cv,0.684 +/- 0.020 (in 3 folds),0.685 +/- 0.026 (in 3 folds),0.494,0.181,0.0,480,0,❗ Missing classes!
lasso_cv,0.676 +/- 0.023 (in 3 folds),0.681 +/- 0.032 (in 3 folds),0.508,0.215,0.0,480,0,✅ no
ridge_cv,0.675 +/- 0.022 (in 3 folds),0.672 +/- 0.018 (in 3 folds),0.494,0.183,0.0,480,0,❗ Missing classes!
linearsvm_ovr,0.674 +/- 0.020 (in 3 folds),0.666 +/- 0.020 (in 3 folds),0.477,0.194,0.0,480,0,✅ no
rf_multiclass,0.673 +/- 0.030 (in 3 folds),0.648 +/- 0.029 (in 3 folds),0.515,0.259,0.0,480,0,✅ no
lasso_multiclass,0.668 +/- 0.020 (in 3 folds),0.655 +/- 0.014 (in 3 folds),0.471,0.223,0.0,480,0,✅ no
xgboost,0.644 +/- 0.020 (in 3 folds),0.629 +/- 0.019 (in 3 folds),0.483,0.218,0.0,480,0,✅ no


# GeneLocus.TCR, TargetObsColumnEnum.disease, metamodel flavor default

,ROC-AUC (weighted OvO) per fold,au-PRC (weighted OvO) per fold,Accuracy global with abstention,MCC global with abstention,abstention_rate,sample_size including abstentions,n_abstentions,missing_classes
ridge_cv,0.956 +/- 0.001 (in 3 folds),0.935 +/- 0.002 (in 3 folds),0.783,0.677,0.002415,414,1,✅ no
elasticnet_cv,0.952 +/- 0.001 (in 3 folds),0.936 +/- 0.003 (in 3 folds),0.795,0.697,0.002415,414,1,✅ no
lasso_multiclass,0.949 +/- 0.008 (in 3 folds),0.942 +/- 0.009 (in 3 folds),0.826,0.755,0.002415,414,1,✅ no
lasso_cv,0.947 +/- 0.008 (in 3 folds),0.934 +/- 0.011 (in 3 folds),0.771,0.659,0.002415,414,1,✅ no
rf_multiclass,0.947 +/- 0.006 (in 3 folds),0.939 +/- 0.007 (in 3 folds),0.773,0.665,0.002415,414,1,✅ no
xgboost,0.944 +/- 0.009 (in 3 folds),0.940 +/- 0.010 (in 3 folds),0.773,0.667,0.002415,414,1,✅ no
linearsvm_ovr,0.944 +/- 0.001 (in 3 folds),0.941 +/- 0.005 (in 3 folds),0.816,0.736,0.002415,414,1,✅ no


# GeneLocus.BCR|TCR, TargetObsColumnEnum.disease, metamodel flavor default

,ROC-AUC (weighted OvO) per fold,au-PRC (weighted OvO) per fold,Accuracy global with abstention,MCC global with abstention,abstention_rate,sample_size including abstentions,n_abstentions,missing_classes
lasso_multiclass,0.983 +/- 0.005 (in 3 folds),0.980 +/- 0.006 (in 3 folds),0.879,0.826,0.016908,414,7,✅ no
elasticnet_cv,0.982 +/- 0.005 (in 3 folds),0.979 +/- 0.006 (in 3 folds),0.884,0.830,0.016908,414,7,✅ no
ridge_cv,0.982 +/- 0.005 (in 3 folds),0.976 +/- 0.008 (in 3 folds),0.877,0.819,0.016908,414,7,✅ no
rf_multiclass,0.981 +/- 0.013 (in 3 folds),0.976 +/- 0.016 (in 3 folds),0.886,0.833,0.016908,414,7,✅ no
linearsvm_ovr,0.980 +/- 0.003 (in 3 folds),0.977 +/- 0.005 (in 3 folds),0.884,0.832,0.016908,414,7,✅ no
lasso_cv,0.976 +/- 0.010 (in 3 folds),0.975 +/- 0.007 (in 3 folds),0.882,0.826,0.016908,414,7,✅ no
xgboost,0.973 +/- 0.008 (in 3 folds),0.971 +/- 0.008 (in 3 folds),0.874,0.817,0.016908,414,7,✅ no


# GeneLocus.BCR|TCR, TargetObsColumnEnum.disease, metamodel flavor isotype_counts_only

,ROC-AUC (weighted OvO) per fold,au-PRC (weighted OvO) per fold,Accuracy global,MCC global,abstention_rate,sample_size including abstentions,n_abstentions,missing_classes
lasso_cv,0.707 +/- 0.008 (in 3 folds),0.713 +/- 0.017 (in 3 folds),0.543,0.261,0.0,414,0,❗ Missing classes!
lasso_multiclass,0.704 +/- 0.023 (in 3 folds),0.697 +/- 0.014 (in 3 folds),0.500,0.268,0.0,414,0,✅ no
rf_multiclass,0.702 +/- 0.019 (in 3 folds),0.684 +/- 0.017 (in 3 folds),0.548,0.302,0.0,414,0,✅ no
elasticnet_cv,0.702 +/- 0.003 (in 3 folds),0.703 +/- 0.017 (in 3 folds),0.539,0.252,0.0,414,0,❗ Missing classes!
linearsvm_ovr,0.695 +/- 0.020 (in 3 folds),0.686 +/- 0.015 (in 3 folds),0.502,0.234,0.0,414,0,✅ no
ridge_cv,0.691 +/- 0.022 (in 3 folds),0.683 +/- 0.023 (in 3 folds),0.522,0.217,0.0,414,0,❗ Missing classes!
xgboost,0.667 +/- 0.013 (in 3 folds),0.662 +/- 0.017 (in 3 folds),0.512,0.256,0.0,414,0,✅ no


In [8]:
# Demographic controlled
for target in [
    TargetObsColumnEnum.disease_all_demographics_present,
]:
    for gene_locus in config.gene_loci_used:
        choose(gene_locus, [target])
    choose(config.gene_loci_used, [target])

# GeneLocus.BCR, TargetObsColumnEnum.disease_all_demographics_present, metamodel flavor default

,ROC-AUC (weighted OvO) per fold,au-PRC (weighted OvO) per fold,Accuracy global with abstention,MCC global with abstention,abstention_rate,sample_size including abstentions,n_abstentions,missing_classes
linearsvm_ovr,0.959 +/- 0.009 (in 3 folds),0.957 +/- 0.010 (in 3 folds),0.821,0.744,0.014286,420,6,✅ no
lasso_multiclass,0.959 +/- 0.008 (in 3 folds),0.957 +/- 0.011 (in 3 folds),0.793,0.707,0.014286,420,6,✅ no
lasso_cv,0.956 +/- 0.009 (in 3 folds),0.954 +/- 0.013 (in 3 folds),0.833,0.756,0.014286,420,6,✅ no
elasticnet_cv,0.955 +/- 0.012 (in 3 folds),0.956 +/- 0.012 (in 3 folds),0.831,0.753,0.014286,420,6,✅ no
rf_multiclass,0.953 +/- 0.010 (in 3 folds),0.950 +/- 0.013 (in 3 folds),0.824,0.743,0.014286,420,6,✅ no
ridge_cv,0.949 +/- 0.011 (in 3 folds),0.951 +/- 0.013 (in 3 folds),0.833,0.758,0.014286,420,6,✅ no
xgboost,0.949 +/- 0.007 (in 3 folds),0.948 +/- 0.014 (in 3 folds),0.817,0.733,0.014286,420,6,✅ no


# GeneLocus.BCR, TargetObsColumnEnum.disease_all_demographics_present, metamodel flavor with_demographics_columns

,ROC-AUC (weighted OvO) per fold,au-PRC (weighted OvO) per fold,Accuracy global with abstention,MCC global with abstention,abstention_rate,sample_size including abstentions,n_abstentions,missing_classes
rf_multiclass,0.965 +/- 0.005 (in 3 folds),0.963 +/- 0.009 (in 3 folds),0.798,0.704,0.014286,420,6,✅ no
elasticnet_cv,0.965 +/- 0.004 (in 3 folds),0.966 +/- 0.007 (in 3 folds),0.814,0.731,0.014286,420,6,✅ no
xgboost,0.959 +/- 0.013 (in 3 folds),0.957 +/- 0.016 (in 3 folds),0.824,0.744,0.014286,420,6,✅ no
lasso_cv,0.956 +/- 0.006 (in 3 folds),0.958 +/- 0.010 (in 3 folds),0.821,0.740,0.014286,420,6,✅ no
ridge_cv,0.955 +/- 0.006 (in 3 folds),0.956 +/- 0.007 (in 3 folds),0.819,0.737,0.014286,420,6,✅ no
lasso_multiclass,0.953 +/- 0.013 (in 3 folds),0.955 +/- 0.010 (in 3 folds),0.807,0.732,0.014286,420,6,✅ no
linearsvm_ovr,0.912 +/- 0.007 (in 3 folds),0.925 +/- 0.011 (in 3 folds),0.783,0.688,0.014286,420,6,✅ no


# GeneLocus.BCR, TargetObsColumnEnum.disease_all_demographics_present, metamodel flavor demographics_regressed_out

...

,ROC-AUC (weighted OvO) per fold,au-PRC (weighted OvO) per fold,Accuracy global,MCC global,abstention_rate,sample_size including abstentions,n_abstentions,missing_classes
rf_multiclass,0.704 +/- 0.036 (in 3 folds),0.686 +/- 0.036 (in 3 folds),0.466,0.250,0.0,358,0,✅ no
xgboost,0.697 +/- 0.032 (in 3 folds),0.691 +/- 0.030 (in 3 folds),0.466,0.199,0.0,358,0,✅ no
lasso_multiclass,0.681 +/- 0.067 (in 3 folds),0.687 +/- 0.059 (in 3 folds),0.338,0.193,0.0,358,0,✅ no
linearsvm_ovr,0.663 +/- 0.028 (in 3 folds),0.678 +/- 0.025 (in 3 folds),0.441,0.144,0.0,358,0,❗ Missing classes!
elasticnet_cv,0.659 +/- 0.007 (in 3 folds),0.679 +/- 0.021 (in 3 folds),0.472,0.110,0.0,358,0,❗ Missing classes!
lasso_cv,0.647 +/- 0.044 (in 3 folds),0.671 +/- 0.049 (in 3 folds),0.472,0.110,0.0,358,0,❗ Missing classes!
ridge_cv,0.640 +/- 0.039 (in 3 folds),0.659 +/- 0.043 (in 3 folds),0.480,0.133,0.0,358,0,❗ Missing classes!


# GeneLocus.BCR|TCR, TargetObsColumnEnum.disease_all_demographics_present, metamodel flavor demographics_only_sex

,ROC-AUC (weighted OvO) per fold,au-PRC (weighted OvO) per fold,Accuracy global,MCC global,abstention_rate,sample_size including abstentions,n_abstentions,missing_classes
rf_multiclass,0.579 +/- 0.023 (in 3 folds),0.547 +/- 0.013 (in 3 folds),0.332,0.110,0.0,358,0,✅ no
linearsvm_ovr,0.573 +/- 0.019 (in 3 folds),0.543 +/- 0.013 (in 3 folds),0.397,0.089,0.0,358,0,❗ Missing classes!
xgboost,0.573 +/- 0.019 (in 3 folds),0.543 +/- 0.013 (in 3 folds),0.461,0.000,0.0,358,0,❗ Missing classes!
lasso_multiclass,0.561 +/- 0.030 (in 3 folds),0.540 +/- 0.016 (in 3 folds),0.332,0.110,0.0,358,0,✅ no
ridge_cv,0.530 +/- 0.052 (in 3 folds),0.517 +/- 0.029 (in 3 folds),0.461,0.000,0.0,358,0,❗ Missing classes!
lasso_cv,0.512 +/- 0.020 (in 3 folds),0.509 +/- 0.016 (in 3 folds),0.461,0.000,0.0,358,0,❗ Missing classes!
elasticnet_cv,0.512 +/- 0.020 (in 3 folds),0.509 +/- 0.016 (in 3 folds),0.461,0.000,0.0,358,0,❗ Missing classes!


# GeneLocus.BCR|TCR, TargetObsColumnEnum.disease_all_demographics_present, metamodel flavor demographics_only_ethnicity_condensed

,ROC-AUC (weighted OvO) per fold,au-PRC (weighted OvO) per fold,Accuracy global,MCC global,abstention_rate,sample_size including abstentions,n_abstentions,missing_classes
ridge_cv,0.792 +/- 0.029 (in 3 folds),0.770 +/- 0.021 (in 3 folds),0.659,0.495,0.0,358,0,❗ Missing classes!
xgboost,0.790 +/- 0.032 (in 3 folds),0.769 +/- 0.021 (in 3 folds),0.665,0.504,0.0,358,0,✅ no
rf_multiclass,0.785 +/- 0.017 (in 3 folds),0.766 +/- 0.014 (in 3 folds),0.564,0.420,0.0,358,0,✅ no
elasticnet_cv,0.780 +/- 0.025 (in 3 folds),0.767 +/- 0.021 (in 3 folds),0.659,0.495,0.0,358,0,❗ Missing classes!
linearsvm_ovr,0.775 +/- 0.023 (in 3 folds),0.761 +/- 0.014 (in 3 folds),0.679,0.533,0.0,358,0,❗ Missing classes!
lasso_cv,0.771 +/- 0.055 (in 3 folds),0.750 +/- 0.053 (in 3 folds),0.659,0.495,0.0,358,0,❗ Missing classes!
lasso_multiclass,0.759 +/- 0.023 (in 3 folds),0.749 +/- 0.016 (in 3 folds),0.640,0.483,0.0,358,0,✅ no


In [9]:
# Demographics from healthy
for target in [
    TargetObsColumnEnum.ethnicity_condensed_healthy_only,
    TargetObsColumnEnum.age_group_healthy_only,
    TargetObsColumnEnum.age_group_binary_healthy_only,
    TargetObsColumnEnum.age_group_pediatric_healthy_only,
    TargetObsColumnEnum.sex_healthy_only,
]:
    for gene_locus in config.gene_loci_used:
        choose(gene_locus, [target])
    choose(config.gene_loci_used, [target])

# GeneLocus.BCR, TargetObsColumnEnum.ethnicity_condensed_healthy_only, metamodel flavor default

,ROC-AUC (weighted OvO) per fold,au-PRC (weighted OvO) per fold,Accuracy global with abstention,MCC global with abstention,abstention_rate,sample_size including abstentions,n_abstentions,missing_classes
elasticnet_cv,0.746 +/- 0.059 (in 3 folds),0.761 +/- 0.030 (in 3 folds),0.602,0.311,0.04712,191,9,❗ Missing classes!
ridge_cv,0.736 +/- 0.043 (in 3 folds),0.745 +/- 0.015 (in 3 folds),0.686,0.436,0.04712,191,9,❗ Missing classes!
linearsvm_ovr,0.721 +/- 0.043 (in 3 folds),0.722 +/- 0.031 (in 3 folds),0.513,0.301,0.04712,191,9,✅ no
rf_multiclass,0.716 +/- 0.081 (in 3 folds),0.734 +/- 0.076 (in 3 folds),0.670,0.446,0.04712,191,9,✅ no
xgboost,0.695 +/- 0.070 (in 3 folds),0.721 +/- 0.056 (in 3 folds),0.607,0.369,0.04712,191,9,✅ no
lasso_cv,0.690 +/- 0.022 (in 3 folds),0.715 +/- 0.020 (in 3 folds),0.681,0.435,0.04712,191,9,❗ Missing classes!
lasso_multiclass,0.686 +/- 0.086 (in 3 folds),0.711 +/- 0.030 (in 3 folds),0.487,0.331,0.04712,191,9,✅ no


# GeneLocus.TCR, TargetObsColumnEnum.ethnicity_condensed_healthy_only, metamodel flavor default

,ROC-AUC (weighted OvO) per fold,au-PRC (weighted OvO) per fold,Accuracy global with abstention,MCC global with abstention,abstention_rate,sample_size including abstentions,n_abstentions,missing_classes
ridge_cv,0.715 +/- 0.041 (in 3 folds),0.734 +/- 0.017 (in 3 folds),0.745,0.562,0.006061,165,1,❗ Missing classes!
elasticnet_cv,0.710 +/- 0.014 (in 3 folds),0.739 +/- 0.033 (in 3 folds),0.739,0.545,0.006061,165,1,❗ Missing classes!
lasso_cv,0.699 +/- 0.017 (in 3 folds),0.727 +/- 0.041 (in 3 folds),0.727,0.513,0.006061,165,1,❗ Missing classes!
lasso_multiclass,0.694 +/- 0.033 (in 3 folds),0.728 +/- 0.017 (in 3 folds),0.461,0.276,0.006061,165,1,✅ no
linearsvm_ovr,0.687 +/- 0.013 (in 3 folds),0.726 +/- 0.012 (in 3 folds),0.503,0.253,0.006061,165,1,✅ no
rf_multiclass,0.668 +/- 0.013 (in 3 folds),0.700 +/- 0.014 (in 3 folds),0.655,0.390,0.006061,165,1,❗ Missing classes!
xgboost,0.630 +/- 0.034 (in 3 folds),0.696 +/- 0.015 (in 3 folds),0.527,0.254,0.006061,165,1,✅ no


# GeneLocus.BCR|TCR, TargetObsColumnEnum.ethnicity_condensed_healthy_only, metamodel flavor default

...

,ROC-AUC (weighted OvO) per fold,au-PRC (weighted OvO) per fold,Accuracy global with abstention,MCC global with abstention,abstention_rate,sample_size including abstentions,n_abstentions,missing_classes
rf_multiclass,0.560 +/- 0.101 (in 3 folds),0.565 +/- 0.138 (in 3 folds),0.482,-0.013,0.026178,191,5,✅ no
linearsvm_ovr,0.514 +/- 0.049 (in 3 folds),0.525 +/- 0.120 (in 3 folds),0.492,0.012,0.026178,191,5,✅ no
lasso_cv,0.500 +/- 0.000 (in 3 folds),0.488 +/- 0.059 (in 3 folds),0.476,-0.031,0.026178,191,5,✅ no
elasticnet_cv,0.500 +/- 0.000 (in 3 folds),0.488 +/- 0.059 (in 3 folds),0.476,-0.031,0.026178,191,5,✅ no
ridge_cv,0.500 +/- 0.000 (in 3 folds),0.488 +/- 0.059 (in 3 folds),0.476,-0.031,0.026178,191,5,✅ no
lasso_multiclass,0.496 +/- 0.059 (in 3 folds),0.514 +/- 0.133 (in 3 folds),0.492,0.011,0.026178,191,5,✅ no
xgboost,0.494 +/- 0.107 (in 3 folds),0.519 +/- 0.153 (in 3 folds),0.466,-0.044,0.026178,191,5,✅ no


# GeneLocus.TCR, TargetObsColumnEnum.sex_healthy_only, metamodel flavor default

,ROC-AUC (weighted OvO) per fold,au-PRC (weighted OvO) per fold,Accuracy global with abstention,MCC global with abstention,abstention_rate,sample_size including abstentions,n_abstentions,missing_classes
rf_multiclass,0.562 +/- 0.037 (in 3 folds),0.646 +/- 0.128 (in 3 folds),0.515,0.076,0.054545,165,9,✅ no
xgboost,0.516 +/- 0.075 (in 3 folds),0.603 +/- 0.129 (in 3 folds),0.497,0.037,0.054545,165,9,✅ no
lasso_cv,0.500 +/- 0.000 (in 3 folds),0.559 +/- 0.056 (in 3 folds),0.424,-0.073,0.054545,165,9,✅ no
elasticnet_cv,0.500 +/- 0.000 (in 3 folds),0.559 +/- 0.056 (in 3 folds),0.424,-0.073,0.054545,165,9,✅ no
ridge_cv,0.500 +/- 0.000 (in 3 folds),0.559 +/- 0.056 (in 3 folds),0.424,-0.073,0.054545,165,9,✅ no
lasso_multiclass,0.487 +/- 0.053 (in 3 folds),0.545 +/- 0.063 (in 3 folds),0.479,0.002,0.054545,165,9,✅ no
linearsvm_ovr,0.482 +/- 0.043 (in 3 folds),0.553 +/- 0.050 (in 3 folds),0.491,0.026,0.054545,165,9,✅ no


# GeneLocus.BCR|TCR, TargetObsColumnEnum.sex_healthy_only, metamodel flavor default

,ROC-AUC (weighted OvO) per fold,au-PRC (weighted OvO) per fold,Accuracy global with abstention,MCC global with abstention,abstention_rate,sample_size including abstentions,n_abstentions,missing_classes
rf_multiclass,0.546 +/- 0.086 (in 3 folds),0.586 +/- 0.118 (in 3 folds),0.473,0.009,0.072727,165,12,✅ no
xgboost,0.542 +/- 0.090 (in 3 folds),0.595 +/- 0.115 (in 3 folds),0.473,0.009,0.072727,165,12,✅ no
lasso_cv,0.500 +/- 0.000 (in 3 folds),0.558 +/- 0.055 (in 3 folds),0.418,-0.067,0.072727,165,12,✅ no
elasticnet_cv,0.500 +/- 0.000 (in 3 folds),0.558 +/- 0.055 (in 3 folds),0.418,-0.067,0.072727,165,12,✅ no
ridge_cv,0.500 +/- 0.000 (in 3 folds),0.558 +/- 0.055 (in 3 folds),0.418,-0.067,0.072727,165,12,✅ no
linearsvm_ovr,0.471 +/- 0.041 (in 3 folds),0.564 +/- 0.061 (in 3 folds),0.473,0.014,0.072727,165,12,✅ no
lasso_multiclass,0.457 +/- 0.055 (in 3 folds),0.545 +/- 0.083 (in 3 folds),0.436,-0.060,0.072727,165,12,✅ no
